In [1]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.gridspec import GridSpec

# Recursive phasor calculation (with sliding window)
def phasor_calculation_recursive(Nw, amp, phase, f, N, f0):
    Nt = N * Nw  # Total number of samples
    fs = f0 * N  # Sampling frequency
    dt = 1 / fs
    t = dt * np.arange(Nt)

    fs1 = N * f
    dt1 = 1 / fs1
    t1 = dt1 * np.arange(Nt)

    x = amp * np.cos(2 * np.pi * f * t + phase)
    x1 = amp * np.cos(2 * np.pi * f * t1 + phase)

    X = np.zeros(Nw, dtype=complex)  # Initialize phasor array

    th = 2 * np.pi / N
    df = f - f0
    w0 = 2 * np.pi * f0
    dw = 2 * np.pi * df
    w = w0 + dw

    if dw == 0:
        P = 1
        Q = 0
    else:
        P = (np.sin(N * (w - w0) * dt / 2) / (N * np.sin((w - w0) * dt / 2))) * np.exp(1j * (N - 1) * (w - w0) * dt / 2)
        Q = (np.sin(N * (w + w0) * dt / 2) / (N * np.sin((w + w0) * dt / 2))) * np.exp(-1j * (N - 1) * (w + w0) * dt / 2)

    # Calculate the first window's phasor
    for n in range(N):
        X[0] += (np.sqrt(2) / N) * x1[n] * np.exp(-1j * n * th)

    # Recursively calculate the subsequent window's phasor
    for k in range(1, Nw):
        X[k] = X[k - 1] + (np.sqrt(2) / N) * (x1[N + k - 1] - x1[k - 1]) * np.exp(-1j * k * th)

    Xb = np.zeros(Nw, dtype=complex)
    for k in range(Nw):
        Xb[k] = P * X[k] * np.exp(1j * k * (w - w0) * dt) + Q * np.conj(X[k]) * np.exp(-1j * k * (w + w0) * dt)

    return Xb

# Tkinter GUI - Modified for responsiveness
class UnifiedPhasorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Unified Phasor Analysis GUI")
        
        # Enable window resizing
        self.root.columnconfigure(0, weight=0)  # Input frame doesn't expand
        self.root.columnconfigure(1, weight=3)  # Plot frame expands
        self.root.rowconfigure(0, weight=1)     # Make the main row expandable
        
        # Initialize variables
        # Enable three-point averaging flag
        self.enable_three_point_avg = False  # Initialize with disabled
        
        # Recursive phasor storage
        self.recursive_phasors = []
        self.sample_index = 0  # Sample index for recursive phasor
        self.total_phase_recursive = 0  # Cumulative phase for recursive phasor
        
        # Non-recursive phasor storage
        self.non_recursive_phasors = []
        self.signal_data = []  # Sliding window data for non-recursive
        self.sample_count = 0  # Sample counter for non-recursive
        
        # Sliding window related variables
        self.window_data = []  # Store all generated data points
        self.current_window_index = 0  # Current window's starting index
        self.is_first_calculation = True  # Flag for first calculation

        # Create input parameter bar - MODIFIED: Reduce width of input frame
        input_frame = tk.Frame(root, width=200)  # Set fixed width to make it smaller
        input_frame.grid(row=0, column=0, padx=5, pady=10, sticky="nw")  # Reduced padx
        input_frame.pack_propagate(False)  # Prevent the frame from resizing based on contents
        
        tk.Label(input_frame, text="Amplitude:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.amplitude_entry = tk.Entry(input_frame, width=10)  # Reduced width
        self.amplitude_entry.grid(row=0, column=1, padx=5, pady=5)
        self.amplitude_entry.insert(0, "10")  # Default value

        tk.Label(input_frame, text="Frequency (Hz):").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.frequency_entry = tk.Entry(input_frame, width=10)
        self.frequency_entry.grid(row=1, column=1, padx=5, pady=5)
        self.frequency_entry.insert(0, "51")  # Default value

        tk.Label(input_frame, text="Nominal Frequency (Hz):").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        self.nominal_frequency_entry = tk.Entry(input_frame, width=10)
        self.nominal_frequency_entry.grid(row=2, column=1, padx=5, pady=5)
        self.nominal_frequency_entry.insert(0, "50")  # Default value

        tk.Label(input_frame, text="Samples per Period:").grid(row=3, column=0, padx=5, pady=5, sticky="w")
        self.samples_entry = tk.Entry(input_frame, width=10)
        self.samples_entry.grid(row=3, column=1, padx=5, pady=5)
        self.samples_entry.insert(0, "20")  # Default value

        tk.Label(input_frame, text="Phase (°):").grid(row=4, column=0, padx=5, pady=5, sticky="w")
        self.phase_entry = tk.Entry(input_frame, width=10)
        self.phase_entry.grid(row=4, column=1, padx=5, pady=5)
        self.phase_entry.insert(0, "0")  # Default value

        # Button frame
        button_frame = tk.Frame(input_frame)
        button_frame.grid(row=5, column=0, columnspan=2, pady=10)

        # Buttons
        self.calculate_button = tk.Button(button_frame, text="Calculate", command=self.calculate_all)
        self.calculate_button.grid(row=0, column=0, padx=5)
        
        self.clear_button = tk.Button(button_frame, text="Clear", command=self.clear_all)
        self.clear_button.grid(row=0, column=1, padx=5)

        # Added three-point average checkbox
        self.three_point_avg_var = tk.BooleanVar()
        self.three_point_avg_checkbox = tk.Checkbutton(input_frame, 
                                                     text="Enable Three-Point Averaging", 
                                                     variable=self.three_point_avg_var,
                                                     command=self.toggle_three_point_avg)
        self.three_point_avg_checkbox.grid(row=8, column=0, columnspan=2, pady=5, sticky="w")

        # Phasor value display area
        phasor_values_frame = tk.LabelFrame(input_frame, text="Phasor Values", padx=5, pady=5)
        phasor_values_frame.grid(row=6, column=0, columnspan=2, sticky="ew", pady=5)
        
        # Recursive phasor values
        tk.Label(phasor_values_frame, text="Recursive Phasor:").grid(row=0, column=0, sticky="w", padx=5, pady=2)
        tk.Label(phasor_values_frame, text="Re:").grid(row=1, column=0, sticky="w", padx=10, pady=2)
        self.recursive_re_label = tk.Label(phasor_values_frame, text="0.0", width=10)
        self.recursive_re_label.grid(row=1, column=1, sticky="w", pady=2)
        
        tk.Label(phasor_values_frame, text="Im:").grid(row=2, column=0, sticky="w", padx=10, pady=2)
        self.recursive_im_label = tk.Label(phasor_values_frame, text="0.0", width=10)
        self.recursive_im_label.grid(row=2, column=1, sticky="w", pady=2)
        
        # Non-recursive phasor values
        tk.Label(phasor_values_frame, text="Non-Recursive Phasor:").grid(row=3, column=0, sticky="w", padx=5, pady=2)
        tk.Label(phasor_values_frame, text="Re:").grid(row=4, column=0, sticky="w", padx=10, pady=2)
        self.non_recursive_re_label = tk.Label(phasor_values_frame, text="0.0", width=10)
        self.non_recursive_re_label.grid(row=4, column=1, sticky="w", pady=2)
        
        tk.Label(phasor_values_frame, text="Im:").grid(row=5, column=0, sticky="w", padx=10, pady=2)
        self.non_recursive_im_label = tk.Label(phasor_values_frame, text="0.0", width=10)
        self.non_recursive_im_label.grid(row=5, column=1, sticky="w", pady=2)
        
        # Status label
        self.status_label = tk.Label(input_frame, text="Ready - Input parameters and click Calculate", fg="blue")
        self.status_label.grid(row=7, column=0, columnspan=2, pady=5)
       
        # Create plot area - MODIFIED FOR RESPONSIVENESS
        self.plot_frame = tk.Frame(root)
        self.plot_frame.grid(row=0, column=1, padx=10, pady=10, sticky="nsew")  # Make plot frame expandable
        self.plot_frame.columnconfigure(0, weight=1)  # Allow plot frame to expand horizontally
        self.plot_frame.rowconfigure(0, weight=1)     # Allow plot frame to expand vertically
        
        # Initialize the figure and canvas but don't create the plots yet
        self.fig = Figure(figsize=(8, 6), dpi=100)  # Initial size, will be adjusted dynamically
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.plot_frame)
        self.canvas_widget = self.canvas.get_tk_widget()
        self.canvas_widget.grid(row=0, column=0, sticky="nsew")  # Make canvas fill the frame
        
        # Bind the resize event to update the plot size
        self.root.bind("<Configure>", self.on_resize)
        
        # Store initial window size to detect significant changes
        self.last_width = self.root.winfo_width()
        self.last_height = self.root.winfo_height()
        
        # Track if plots need redrawing after resize
        self.resize_timer = None
    
    def toggle_three_point_avg(self):
        """Toggle three-point averaging algorithm"""
        self.enable_three_point_avg = self.three_point_avg_var.get()
        status = "enabled" if self.enable_three_point_avg else "disabled"
        self.status_label.config(text=f"Three-point averaging {status}", 
                               fg="green" if self.enable_three_point_avg else "blue")
        
    def on_resize(self, event):
        """Handle window resize events"""
        # Only respond to root window events to avoid excessive redrawing
        if event.widget == self.root:
            # Cancel any previous timer
            if self.resize_timer:
                self.root.after_cancel(self.resize_timer)
            
            # Start a new timer to update the plot after resizing stops
            self.resize_timer = self.root.after(200, self.update_plot_size)
    
    def update_plot_size(self):
        """Update the plot size after the window has been resized"""
        # Check if we have actual plots to redraw
        if hasattr(self, 'ax_polar1') and not self.is_first_calculation:
            # Calculate new figsize based on frame size
            width = self.plot_frame.winfo_width() / 100  # Convert to inches (approximate)
            height = self.plot_frame.winfo_height() / 100
            
            # Ensure minimum size
            width = max(width, 4)
            height = max(height, 3)
            
            # Update figure size
            self.fig.set_size_inches(width, height)
            
            # Adjust the subplot layout
            self.fig.tight_layout(pad=1.5)
            
            # Redraw the canvas
            self.canvas.draw()
            
            # Reset the timer
            self.resize_timer = None
    
    def clear_all(self):
        """Clear all charts and data"""
        self.fig.clear()
        
        # Reset all data
        self.recursive_phasors = []
        self.total_phase_recursive = 0
        self.sample_index = 0
        
        self.non_recursive_phasors = []
        self.signal_data = []
        self.sample_count = 0
        
        self.window_data = []
        self.current_window_index = 0
        self.is_first_calculation = True
        
        # Reset phasor value display
        self.recursive_re_label.config(text="0.0")
        self.recursive_im_label.config(text="0.0")
        self.non_recursive_re_label.config(text="0.0")
        self.non_recursive_im_label.config(text="0.0")
        
        self.status_label.config(text="Cleared - Ready for new calculations")
        self.canvas.draw()
        
        # Reset three-point averaging
        self.three_point_avg_var.set(False)
        self.enable_three_point_avg = False
    
    def prefill_window(self):
        """Prefill signal window, generate N valid signal samples"""
        amp = float(self.amplitude_entry.get())
        f0 = float(self.nominal_frequency_entry.get())
        N = int(self.samples_entry.get())
        phase = float(self.phase_entry.get())
        
        # Clear existing data
        self.signal_data = []
        self.sample_count = 0
        
        # Sampling frequency and time interval
        fs = f0 * N
        dt = 1 / fs
        
        # Pre-generate N signal samples
        for i in range(N):
            sample_time = self.sample_count * dt
            self.sample_count += 1
            sample = amp * np.cos(2 * np.pi * f0 * sample_time + np.deg2rad(phase))
            self.signal_data.append(sample)
    
    def phasor_calculation_non_recursive(self, amp, phase, f0, N):
        """Calculate non-recursive phasor, based on sliding window"""
        fs = f0 * N  # Sampling frequency
        dt = 1 / fs   # Sampling time interval
        
        # Use sample counter to generate continuous time points
        new_sample_time = self.sample_count * dt
        self.sample_count += 1
        
        # Generate new sample
        new_sample = amp * np.cos(2 * np.pi * f0 * new_sample_time + np.deg2rad(phase))  

        # Sliding window management
        if len(self.signal_data) >= N:
            self.signal_data.pop(0)  # Remove the earliest sample point
        self.signal_data.append(new_sample)  # Add new sample point

        # Zero-pad to match N length
        signal_array = np.array(self.signal_data)
        if len(signal_array) < N:
            signal_array = np.pad(signal_array, (N - len(signal_array), 0), mode='constant')

        # Calculate phasor
        theta = 2 * np.pi / N  # Angle increment
        X = np.sum((np.sqrt(2) / N) * signal_array * np.exp(-1j * np.arange(N) * theta))

        return X
    
    def calculate_recursive_phasor(self):
        """Calculate recursive phasor"""
        amp = float(self.amplitude_entry.get())
        f = float(self.frequency_entry.get())
        f0 = float(self.nominal_frequency_entry.get())
        N = int(self.samples_entry.get())
        phase = np.deg2rad(float(self.phase_entry.get()))
        Nw = 2  # Fixed at 2
        
        # Calculate phase increment (per sample point)
        sample_time = 1 / (f0 * N)  # Time for each sample point
        df = f - f0
        theoretical_increment = 2 * np.pi * df * sample_time  # Theoretical phase increment per sample point
        
        # Cumulative phase
        if self.sample_index == 0:
            # First calculation, initialize phase
            Xb = phasor_calculation_recursive(Nw, amp, phase, f, N, f0)
            current_phasor = Xb[0]  # Use the first phasor
            self.total_phase_recursive = np.angle(current_phasor)
        else:
            # Subsequent calculations, accumulate phase
            self.total_phase_recursive += theoretical_increment
        
        # Create phasor with cumulative phase
        new_phasor = amp * np.exp(1j * self.total_phase_recursive)
        
        # Store new phasor and update index
        self.recursive_phasors.append(new_phasor)
        self.sample_index += 1  # Slide one sample point
        
        return new_phasor, df, theoretical_increment

    def calculate_non_recursive_phasor(self):
        """Calculate non-recursive phasor"""
        amp = float(self.amplitude_entry.get())
        f0 = float(self.nominal_frequency_entry.get())
        N = int(self.samples_entry.get())
        phase = float(self.phase_entry.get())
        
        # Check if window is already filled, if not, pre-fill
        if len(self.signal_data) < N:
            self.prefill_window()
        
        # Calculate new phasor
        new_phasor = self.phasor_calculation_non_recursive(amp, phase, f0, N)
        
        # Store the calculated phasor
        self.non_recursive_phasors.append(new_phasor)
        
        return new_phasor
    
    def generate_data_if_needed(self):
        """Generate data as needed, supporting infinite sliding"""
        # Get input parameters
        V_m = float(self.amplitude_entry.get())
        f = float(self.frequency_entry.get())
        f_0 = float(self.nominal_frequency_entry.get())
        N = int(self.samples_entry.get())
        phi = float(self.phase_entry.get()) * np.pi / 180

        # If first calculation, initialize parameters and generate initial dataset
        if self.is_first_calculation:
            # Calculate sampling interval
            Delta_t = 1 / (N * f_0)
            
            # Initially generate 50 periods of data (adjustable as needed)
            n_periods = 50
            total_samples = n_periods * N
            t = np.arange(0, total_samples * Delta_t, Delta_t)
            
            # Generate signal data
            self.window_data = V_m * np.cos(2 * np.pi * f * t + phi)
            
            # Store parameters for subsequent calculations
            self.params = {
                'V_m': V_m,
                'f': f,
                'f_0': f_0,
                'N': N,
                'phi': phi,
                'Delta_t': Delta_t
            }
            
            self.is_first_calculation = False
            return True
        
        # Check if we need to generate more data
        if self.current_window_index + self.params['N'] * 2 > len(self.window_data):
            # Need to generate more data, add 20 periods each time
            n_additional_periods = 20
            additional_samples = n_additional_periods * self.params['N']
            
            # Calculate new data start time
            start_time = len(self.window_data) * self.params['Delta_t']
            end_time = start_time + additional_samples * self.params['Delta_t']
            t_new = np.arange(start_time, end_time, self.params['Delta_t'])
            
            # Generate new data points
            new_data = self.params['V_m'] * np.cos(2 * np.pi * self.params['f'] * t_new + self.params['phi'])
            
            # Add new data to existing data
            self.window_data = np.append(self.window_data, new_data)
            
            return True
        
        return False
    
    def calculate_all(self):
        """Calculate and display all charts - MODIFIED for responsiveness"""
        try:
            # Clear current charts
            self.fig.clear()
            
            # Set figure size based on current frame size
            width = self.plot_frame.winfo_width() / 100  # Convert to inches (approximate)
            height = self.plot_frame.winfo_height() / 100
            
            # Ensure minimum size
            width = max(width, 4)
            height = max(height, 3)
            
            self.fig.set_size_inches(width, height)
            
            # Calculate recursive phasor
            recursive_phasor, df, theoretical_increment = self.calculate_recursive_phasor()
            
            # Calculate non-recursive phasor
            non_recursive_phasor = self.calculate_non_recursive_phasor()
            
            # Check if window data needs to be generated
            self.generate_data_if_needed()
            
            # Get parameters
            N = int(self.samples_entry.get())
            f_0 = float(self.nominal_frequency_entry.get())
            Delta_t = 1 / (N * f_0)
            f = float(self.frequency_entry.get())
            
            # Get current window data
            current_window = self.window_data[self.current_window_index:self.current_window_index + N]
            
            # Calculate time axis for current window
            t_window = np.arange(self.current_window_index, self.current_window_index + N) * Delta_t
            
            # Calculate angular frequency
            omega_0 = 2 * np.pi * f_0  
            omega = 2 * np.pi * f  
            
            # Calculate sampling interval
            duration = 2 / f  
            t = np.linspace(t_window[0], t_window[0] + duration, int(N * 2))  
            sampled_t = t_window
            
            # Generate signal
            signal = float(self.amplitude_entry.get()) * np.cos(2 * np.pi * f * t + np.deg2rad(float(self.phase_entry.get())))
            sampled_signal = current_window

            # FFT spectrum
            N_fft = len(sampled_signal)
            signal_fft = np.fft.fft(sampled_signal)
            signal_fft = np.fft.fftshift(signal_fft)  
            f_plot = np.linspace(-1/(2*Delta_t), 1/(2*Delta_t), N_fft)  

            # Computing recursive phasors
            n_end = 200  
            # Use current window's start time
            start_time = self.current_window_index * Delta_t
            t_true = np.arange(start_time, start_time + n_end * Delta_t, Delta_t)
            x_n_true = np.sqrt(2) * float(self.amplitude_entry.get()) * np.cos(omega * t_true + np.deg2rad(float(self.phase_entry.get())))

            def get_phasor(x_w, N):
                X_N_v = x_w * np.exp(-1j * np.arange(N) * (2 * np.pi / N))
                return (np.sqrt(2) / N) * np.sum(X_N_v)

            X_hat_0 = get_phasor(x_n_true[:N], N)
            X_hat_prev = X_hat_0
            X_hat_v = [X_hat_prev]

            j = N
            while j < n_end - N + 1:
                r = j - N
                X_hat_i = X_hat_prev + (np.sqrt(2) / N) * (x_n_true[N + r] - x_n_true[r]) * np.exp(-1j * r * (2 * np.pi / N))
                X_hat_v.append(X_hat_i)
                X_hat_prev = X_hat_i
                j += 1

            # Convert to NumPy array
            X_hat_v = np.array(X_hat_v)
            
            # Store original X_hat_v for comparison
            X_hat_v_original = X_hat_v.copy()
            
            # Apply three-point averaging to X_hat_v if enabled
            if self.enable_three_point_avg and len(X_hat_v) > 1:
                N = int(self.samples_entry.get())
                k60 = round(N / 6)  # 60-degree phase shift point (1/6 of a cycle)
                k120 = round(N / 3)  # 120-degree phase shift point (1/3 of a cycle)
                
                # Create a filtered version of X_hat_v
                X_hat_v_filtered = np.copy(X_hat_v)
                
                # Only apply averaging when we have enough points
                if len(X_hat_v) > k120 + 1:
                    # Apply three-point averaging to each element after first k120 points
                    for i in range(k120, len(X_hat_v)):
                        X_hat_v_filtered[i] = (X_hat_v[i] + X_hat_v[i-k60] + X_hat_v[i-k120]) / 3
                    
                    # Store the filtered data
                    X_hat_v_filtered_for_display = X_hat_v_filtered.copy()
                    # Use the filtered data for subsequent calculations
                    X_hat_v = X_hat_v_filtered
            else:
                # If not enabled, both are the same
                X_hat_v_filtered_for_display = X_hat_v
                        
            # Create a more responsive grid spec
            gs = GridSpec(3, 2, figure=self.fig, height_ratios=[1.5, 1, 1])
                        
            # Polar plots on top row
            self.ax_polar1 = self.fig.add_subplot(gs[0, 0], projection='polar')
            self.ax_polar2 = self.fig.add_subplot(gs[0, 1], projection='polar')
                        
            # Time domain in middle left
            self.ax_time = self.fig.add_subplot(gs[1, 0])
                        
            # FFT spectrum in middle right
            self.ax_fft = self.fig.add_subplot(gs[1, 1])
                        
            # Recursive phasor magnitude in bottom left
            self.ax_mag = self.fig.add_subplot(gs[2, 0])
                        
            # Recursive phasor phase in bottom right
            self.ax_phase = self.fig.add_subplot(gs[2, 1])
                        
            # Set chart titles
            self.ax_polar1.set_title("Recursive Phasor", fontsize=12)
            self.ax_polar2.set_title("Non-Recursive Phasor", fontsize=12)
                        
            # Draw recursive phasors - historical phasors in black
            for i, phasor in enumerate(self.recursive_phasors[:-1]):
                magnitude = np.abs(phasor)
                angle = np.angle(phasor)
                # Use black, smaller dots and thin lines
                self.ax_polar1.plot([0, angle], [0, magnitude], marker='.', markersize=3, linestyle='-', 
                                     linewidth=0.8, color='black', alpha=0.7)
                        
            # Draw latest recursive phasor in red
            if self.recursive_phasors:
                latest_recursive = self.recursive_phasors[-1]
                magnitude = np.abs(latest_recursive)
                angle = np.angle(latest_recursive)
                # Use red, smaller dots and thick lines
                self.ax_polar1.plot([0, angle], [0, magnitude], marker='.', markersize=4, linestyle='-', 
                                     linewidth=1.5, color='red')
                            
                # Add label for latest recursive phasor
                angle_deg = np.degrees(self.total_phase_recursive) % 360
                self.ax_polar1.text(angle, magnitude * 1.1, f"{angle_deg:.1f}°", 
                                     horizontalalignment='center', verticalalignment='bottom', color='red', fontsize=8)
                        
            # Draw non-recursive phasors - historical phasors in black
            for i, phasor in enumerate(self.non_recursive_phasors[:-1]):
                magnitude = np.abs(phasor)
                angle = np.angle(phasor)
                # Use black, smaller dots and thin lines
                self.ax_polar2.plot([0, angle], [0, magnitude], marker='.', markersize=3, linestyle='-', 
                                     linewidth=0.8, color='black', alpha=0.7)
                        
            # Draw latest non-recursive phasor in red
            if self.non_recursive_phasors:
                latest_non_recursive = self.non_recursive_phasors[-1]
                magnitude = np.abs(latest_non_recursive)
                angle = np.angle(latest_non_recursive)
                # Use red, smaller dots and thick lines
                self.ax_polar2.plot([0, angle], [0, magnitude], marker='.', markersize=3, linestyle='-', 
                                     linewidth=1.0, color='red')
                            
                # Add label for latest non-recursive phasor
                angle_deg = np.degrees(angle)
                self.ax_polar2.text(angle, magnitude * 1.1, f"{angle_deg:.1f}°", 
                                     horizontalalignment='center', verticalalignment='bottom', color='red', fontsize=8)
                        
            # Set chart range to ensure proper display
            max_recursive_magnitude = max([np.abs(p) for p in self.recursive_phasors]) if self.recursive_phasors else 1
            max_non_recursive_magnitude = max([np.abs(p) for p in self.non_recursive_phasors]) if self.non_recursive_phasors else 1
            self.ax_polar1.set_rmax(max_recursive_magnitude * 1.2)
            self.ax_polar2.set_rmax(max_non_recursive_magnitude * 1.2)
                        
            # Adjust tick label sizes for better visibility on smaller screens
            self.ax_polar1.tick_params(labelsize=10)
            self.ax_polar2.tick_params(labelsize=10)
                        
            # Draw time domain signal chart
            self.ax_time.plot(t, signal, label="Continuous Signal", color="b")
            self.ax_time.stem(sampled_t, sampled_signal, linefmt="r-", markerfmt="ro", basefmt="k")
            self.ax_time.set_xlabel("Time (s)", fontsize=9)
            self.ax_time.set_ylabel("Amplitude", fontsize=9)
            self.ax_time.set_title(f"Time Domain Signal (Window {self.current_window_index//N + 1})", fontsize=10)
            self.ax_time.legend(fontsize=8)
            self.ax_time.grid(True)
            self.ax_time.tick_params(labelsize=8)
                        
            # Draw FFT spectrum chart
            self.ax_fft.plot(f_plot, np.abs(signal_fft), color="g")
            self.ax_fft.set_xlabel("Frequency (Hz)", fontsize=9)
            self.ax_fft.set_ylabel("Magnitude", fontsize=9)
            self.ax_fft.set_title("Frequency Spectrum", fontsize=10)
            self.ax_fft.grid(True)
            self.ax_fft.tick_params(labelsize=8)
                        
            # Find max and min value indices
            max_idx = np.argmax(np.abs(signal_fft))  # Max value index
            min_idx = np.argmin(np.abs(signal_fft))  # Min value index
                        
            # Get max and min values
            max_freq = f_plot[max_idx]
            max_value = np.abs(signal_fft[max_idx])
            min_freq = f_plot[min_idx]
            min_value = np.abs(signal_fft[min_idx])
                        
            # Annotate max and min values on the chart
            self.ax_fft.text(max_freq, max_value, f'{max_value:.2f}', fontsize=8, color='red', ha='left', va='bottom')
            self.ax_fft.text(min_freq, min_value, f'{min_value:.2f}', fontsize=8, color='blue', ha='left', va='top')
                        
            # Draw red and blue dots at max and min value positions
            self.ax_fft.plot(max_freq, max_value, 'ro', markersize=4)  # Red dot (max value)
            self.ax_fft.plot(min_freq, min_value, 'bo', markersize=4)  # Blue dot (min value)
                        
            # Draw recursive phasor magnitude chart
            self.ax_mag.plot(np.abs(X_hat_v_original), label="Original", color="b", marker='*', markersize=4, linestyle='-', alpha=0.7)
                        
            if self.enable_three_point_avg:
                self.ax_mag.plot(np.abs(X_hat_v_filtered_for_display), label="Three-Point Avg", color="green", linewidth=2)
                self.ax_mag.set_title(f"Phasor Magnitude (Window {self.current_window_index//N + 1})", fontsize=10)
            else:
                self.ax_mag.set_title(f"Phasor Magnitude (Window {self.current_window_index//N + 1})", fontsize=10)
            
            self.ax_mag.set_xlabel("Sample Index", fontsize=9)
            self.ax_mag.set_ylabel("Magnitude", fontsize=9)
            self.ax_mag.legend(fontsize=8)
            self.ax_mag.grid(True)
            self.ax_mag.tick_params(labelsize=8)
            
            # Find the maximum and minimum value indices (using currently displayed data)
            data_for_minmax = X_hat_v if self.enable_three_point_avg else X_hat_v_original
            max_idx = np.argmax(np.abs(data_for_minmax))  # Max value index
            min_idx = np.argmin(np.abs(data_for_minmax))  # Min value index
            
            # Get the maximum and minimum values
            max_value = np.abs(data_for_minmax[max_idx])
            min_value = np.abs(data_for_minmax[min_idx])
            
            # Mark the maximum and minimum values ​​on the graph
            self.ax_mag.text(max_idx, max_value, f'{max_value:.2f}', fontsize=8, color='red', ha='left', va='bottom')
            self.ax_mag.text(min_idx, min_value, f'{min_value:.2f}', fontsize=8, color='blue', ha='left', va='top')
            
            # Draw red and blue points at the maximum and minimum values
            self.ax_mag.plot(max_idx, max_value, 'ro', markersize=4)  # Red dot (max value)
            self.ax_mag.plot(min_idx, min_value, 'bo', markersize=4)  # Blue dot (min value)
            
            # Draw recursive phasor phase chart
            self.ax_phase.plot(np.angle(X_hat_v_original) * 180 / np.pi, label="Original", color="r", linewidth=1.5, alpha=0.7)
            
            if self.enable_three_point_avg:
                self.ax_phase.plot(np.angle(X_hat_v_filtered_for_display) * 180 / np.pi, label="Three-Point Avg", color="darkgreen", linewidth=2)
                self.ax_phase.set_title(f"Phasor Phase (Window {self.current_window_index//N + 1})", fontsize=10)
            else:
                self.ax_phase.set_title(f"Phasor Phase (Window {self.current_window_index//N + 1})", fontsize=10)
            
            self.ax_phase.set_xlabel("Sample Index", fontsize=9)
            self.ax_phase.set_ylabel("Phase (degrees)", fontsize=9)
            self.ax_phase.legend(fontsize=8)
            self.ax_phase.grid(True)
            self.ax_phase.tick_params(labelsize=8)
                        
            # Apply tight layout with fixed padding to prevent overlap
            self.fig.tight_layout(pad=1.5)
                        
            # Update phasor value display
            if self.recursive_phasors:
                latest_recursive = self.recursive_phasors[-1]
                self.recursive_re_label.config(text=f"{latest_recursive.real:.4f}")
                self.recursive_im_label.config(text=f"{latest_recursive.imag:.4f}")
                        
            if self.non_recursive_phasors:
                latest_non_recursive = self.non_recursive_phasors[-1]
                self.non_recursive_re_label.config(text=f"{latest_non_recursive.real:.4f}")
                self.non_recursive_im_label.config(text=f"{latest_non_recursive.imag:.4f}")
                        
            # Update window index for next calculation
            self.current_window_index += N
                        
            # Update status
            window_num = self.current_window_index // N
            if self.enable_three_point_avg:
                self.status_label.config(text=f"Window {window_num} - Three-Point Averaging Enabled", fg="green")
            else:
                self.status_label.config(text=f"Window {window_num} - All plots updated")
                        
            # Display charts
            self.canvas.draw()
                        
        except ValueError:
                messagebox.showerror("Input Error", "Please enter valid numeric values.")
        except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def plot_signal(self):
        """Draw signal analysis charts - kept for compatibility, not actually used"""
        pass

    def plot_polar_phasors(self):
        """Draw polar phasor charts - kept for compatibility, not actually used"""
        pass


# Start Tkinter
if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("1280x800")  # Set initial window size
    app = UnifiedPhasorApp(root)
    root.mainloop()

ModuleNotFoundError: No module named 'numpy'